In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import math
import re

print(f"Selenium version: {selenium.__version__}")

Selenium version: 4.12.0


In [2]:
product_df = pd.read_csv("./데이터 수집/상품 목록 (리뷰 100개 이상)_오름차순.csv")
product_df

,product_id,product_url,product_name,review_num
0,1000498619,https://www.kurly.com/goods/1000498619,[한반] 양곰탕,101
1,1000356831,https://www.kurly.com/goods/1000356831,[호밍스] 얼큰곤이알탕 400g,102
2,1000498621,https://www.kurly.com/goods/1000498621,[한반] 내장탕,110
3,1000659319,https://www.kurly.com/goods/1000659319,[리북방] 이북식 백순대국,118
4,1000331934,https://www.kurly.com/goods/1000331934,[풀무원] 베트남 쌀국수 Pho 4인분,135
...,...,...,...,...
264,5051186,https://www.kurly.com/goods/5051186,[강남면옥] 갈비탕,44921
265,1000040783,https://www.kurly.com/goods/1000040783,[바른식] 강릉식 짬뽕순두부 찌개,47511
266,5056783,https://www.kurly.com/goods/5056783,[강창구찹쌀진순대] 찹쌀 진순대국,47721
267,5042765,https://www.kurly.com/goods/5042765,[외할머니댁] 차돌 된장찌개,50291


### 크롤링 코드

In [4]:
def extract_help_num():
    help_num_list = driver.find_elements(By.CSS_SELECTOR, 'button.css-g3a39p.e198bwfo1 > span:nth-child(2)')
    help_num = []

    for num in help_num_list :
        match = re.search(r'\d+', num.text)
    
        if match:
            help_num.append(int(match.group()))
        else:
            help_num.append(0)
            
    return help_num




def get_reviews(product_id):
    df = pd.DataFrame(columns=['product_id', 'product_type', 'date', 'help_num', 'review'])
    product_url = 'https://www.kurly.com/goods/'+ str(product_id)
    
    driver.get(product_url)
    clicks = 0
    
    try:
        while True:
            review = list(map(lambda x : x.text, driver.find_elements(By.CSS_SELECTOR, 'p.css-y49dcn.e36z05c13')))
            product_type = list(map(lambda x : x.text.split(']')[-1].strip(), driver.find_elements(By.CSS_SELECTOR, 'h3.css-11q4ylb.e36z05c12')))
            date = list(map(lambda x : x.text, driver.find_elements(By.CSS_SELECTOR, 'span.css-14kcwq8.e36z05c0')))
            help_num = extract_help_num()
                
            
            df = df.append(pd.DataFrame({'product_id' : [product_id] * len(date),
                                        'product_type': product_type,
                                        'date' : date,
                                        'help_num' : help_num,
                                        'review' : review}))
            
                
            
            button = driver.find_elements(By.CSS_SELECTOR, 'button.css-1orps7k.ebs5rpx0')[0]
            if not button.get_attribute('disabled'):
                button.click()
                clicks+=1
            else:
                break
            time.sleep(1.7)
        
        return df

            
    except Exception as e:
        print("e :", e, "clicks :", clicks, 'product_id :', product_id)
        return df

In [5]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

In [15]:
review_df = pd.DataFrame(columns=['product_id', 'product_type', 'date', 'help_num', 'review'])

In [16]:
for product_id in [5119615, 1000047929]:
    review_df = pd.concat([review_df, get_reviews(product_id)], axis=0)

In [23]:
review_df

,product_id,product_type,date,help_num,review
0,5097092,한우사골 양지곰탕,2024.06.28,9,간편식으로 먹으려고 한우사골 양지곰탕을 구매해 보았습니다.\n한우사골 양지곰탕의 조...
1,5097092,한우사골 양지곰탕,2023.08.01,36,밥 잘 안챙겨먹고 요리도 귀찮고 하기 힘든 스타일 이시라면 무조건 구매해 두세요.....
2,5097092,한우사골 양지곰탕,2023.04.08,36,한우사골 양지곰탕으로 \n사골떡만두칼국수🍜 만들어 먹었어요 ~ \n\n국물이 진하고...
3,5097092,한우사골 양지곰탕,2024.08.31,0,일단 포장이 이렇게 찢어져서 왔음. 좀놀랬지만 귀찮아서 그냥 먹으려고했는데 \n끓이...
4,5097092,한우사골 양지곰탕,2024.08.30,0,아침에 후루룩 말아주기 좋아요
...,...,...,...,...,...
0,5132868,부산깡통시장 유부주머니,2022.05.12,0,나들이용으로 구입했어오\n맛있길 기도합니다 ㅎㅎ
1,5132868,부산깡통시장 유부주머니,2022.05.11,0,신랑이 좋아해서 구매합니다
2,5132868,부산깡통시장 유부주머니,2022.05.11,0,유부주머니 넘 좋아하는데 기대되요
3,5132868,부산깡통시장 유부주머니,2022.05.11,0,부산 여행 생각나서 구매해봤어요


In [24]:
review_df['product_id'].unique()

array([5097092, 5132868], dtype=object)

In [9]:
product_df.iloc[220:240]

,product_id,product_url,product_name,review_num
220,5049304,https://www.kurly.com/goods/5049304,[프레시메이드] 설렁탕,8538
221,5031816,https://www.kurly.com/goods/5031816,[돌쇠] 한우사골 곰탕,8736
222,1000088626,https://www.kurly.com/goods/1000088626,[사미헌] 우거지 갈비탕,8750
223,5070615,https://www.kurly.com/goods/5070615,[진실된손맛] 소곱창전골,8786
224,5049686,https://www.kurly.com/goods/5049686,[미자언니네] 백암 순대국 (2인분),8816
225,5052864,https://www.kurly.com/goods/5052864,[외할머니댁] 차돌 청국장,9287
226,5116405,https://www.kurly.com/goods/5116405,[피코크] 쟌슨빌 부대찌개,9531
227,5026469,https://www.kurly.com/goods/5026469,[사미헌] 꼬리곰탕,9604
228,5097094,https://www.kurly.com/goods/5097094,[진실된손맛] 소고기 샤브탕,9657
229,5131413,https://www.kurly.com/goods/5131413,[진실된손맛] 맑은 나주식곰탕,10183


### 중간에 끊겼을때

In [ ]:
while True:
    button = driver.find_elements(By.CSS_SELECTOR, 'button.css-1orps7k.ebs5rpx0')[0]
    button.click()
    time.sleep(1)

In [43]:
df = pd.DataFrame(columns=['product_id', 'product_type', 'date', 'help_num', 'review'])

In [44]:
try:
    while True:
        review = list(map(lambda x : x.text, driver.find_elements(By.CSS_SELECTOR, 'p.css-y49dcn.e36z05c13')))
        product_type = list(map(lambda x : x.text.split(']')[-1].strip(), driver.find_elements(By.CSS_SELECTOR, 'h3.css-11q4ylb.e36z05c12')))
        date = list(map(lambda x : x.text, driver.find_elements(By.CSS_SELECTOR, 'span.css-14kcwq8.e36z05c0')))
        help_num = extract_help_num()


        df = df.append(pd.DataFrame({'product_id' : [product_id] * len(date),
                                    'product_type': product_type,
                                    'date' : date,
                                    'help_num' : help_num,
                                    'review' : review}))



        button = driver.find_elements(By.CSS_SELECTOR, 'button.css-1orps7k.ebs5rpx0')[0]
        if not button.get_attribute('disabled'):
            button.click()
        else:
            break
        time.sleep(1.7)


except Exception as e:
    print("e :", e, 'product_id :', product_id)

In [45]:
df

,product_id,product_type,date,help_num,review
0,5132868,김치 우동 어묵 전골 (리뉴얼),2023.11.05,40,진짜 사고 싶었던 삼진어묵 김치 어묵 전골입니다. \n컬리에서 일일특가로 완전 득템...
1,5132868,김치 우동 어묵 전골 (리뉴얼),2023.07.24,50,"캠핑가서 진짜 맛있게 먹었어요.\n\n식사용으로는 양이 부족할 것 같아서, 물놀이 ..."
2,5132868,김치 우동 어묵 전골 (리뉴얼),2023.06.19,39,저의 구매 목적은 뚜벅이 여행을 위해..짐이 많으면 안되니\n제일 간편하게 가져갈 ...
3,5132868,김치 우동 어묵 전골 (리뉴얼),2024.09.01,0,이제우동을 먹을 날씨가 돠어서 시켜봤어요. 일식 돈까스 집에 가면 김치우동 같이 시...
4,5132868,김치 우동 어묵 전골 (리뉴얼),2024.09.01,0,첫구매여\n해장으로 매운고추가루 조굼이랑\n우동사리 한개 더 넣서\n먹었어요 맛있게...
...,...,...,...,...,...
3,5132868,김치 우동 어묵 전골,2022.10.14,0,김치우동 어묵 전골 삼진어묵 맛있으니 맛있을것같아요
4,5132868,김치 우동 어묵 전골,2022.10.14,0,선택했어요\n실수로 3개시켰지만 기대중입니다
5,5132868,김치 우동 어묵 전골,2022.10.13,0,처음 주문 해보아요
6,5132868,김치 우동 어묵 전골,2022.10.13,0,타 브랜거 먹다 세일하길래 주문해요


In [14]:
저장 = review_df
저장

,product_id,product_type,date,help_num,review
0,5049304,설렁탕,2024-06-07,0,프레쉬 메이드 설렁탕😌\n예전에 한 번 먹어보고\n괜찮다고 생각해서 재구매 해봤어요...
1,5049304,설렁탕,2021-05-21,20,예전에 프레시메이드 나주곰탕을 먹어봤었는데 \n너무 괜찮았었거든요~ \n그래서 설렁...
2,5049304,설렁탕,2024-08-23,0,어머니가 가장 좋아하셔서 자주 구매하고 있어요. 고기도 넉넉해요.
3,5049304,설렁탕,2024-08-23,0,비상용으로 쟁여놔요 \n아이들이 좋아해요
4,5049304,설렁탕,2024-08-22,0,한끼 식사로딱이에요 아이들이 좋아해요
...,...,...,...,...,...
182165,5097307,대파 육개장,2021-11-28,0,혼자서 두번 나눠먹을 양은 돼요.\n칼칼하고 진한 국물이라 \n식사용으로도 좋고 안...
182166,5097307,대파 육개장,2021-11-28,0,맛있어서 자주 구매
182167,5097307,대파 육개장,2021-11-26,0,맛 있어서 자주 구매하다가 오늘은 오랜만에 사게되네요
182168,5097307,대파 육개장,2021-11-26,0,육개장 좋아해서 구매해서 아침부터 \n끓여서 먹어봤는데 역시 맛있어요 성공!\n할인...


In [41]:
앞쪽 = 저장[저장['product_id'] == 1000047929]
앞쪽

,product_id,product_type,date,help_num,review
99944,1000047929,김치 우동 어묵 전골 (리뉴얼),2023-11-05,40,진짜 사고 싶었던 삼진어묵 김치 어묵 전골입니다. \n컬리에서 일일특가로 완전 득템...
99945,1000047929,김치 우동 어묵 전골 (리뉴얼),2023-07-24,50,"캠핑가서 진짜 맛있게 먹었어요.\n\n식사용으로는 양이 부족할 것 같아서, 물놀이 ..."
99946,1000047929,김치 우동 어묵 전골 (리뉴얼),2023-06-19,39,저의 구매 목적은 뚜벅이 여행을 위해..짐이 많으면 안되니\n제일 간편하게 가져갈 ...


In [39]:
new = pd.concat([앞쪽, df])
new

,product_id,product_type,date,help_num,review
91893,5119615,금돼지식당 김치찌개,2023-04-03,100,예전에 이마트갔다가 본건데 살까말까 고민하다가 안샀는데\n컬리에도 있길래 보고 갑자...
91894,5119615,금돼지식당 김치찌개,2023-03-07,44,돼지고기 김치찌개가 생각나는 날이어서 끓여보았어요 ~ \n워낙 유명한 금돼지식당 김...
91895,5119615,금돼지식당 김치찌개,2024-08-25,0,제 입맛엔 비계많은 김치찌개 느낌이었어요 ㅜ
91896,5119615,금돼지식당 김치찌개,2024-08-25,0,재재재주문입미다..굿
91897,5119615,금돼지식당 김치찌개,2024-08-25,0,세알하면 무조건 쟁여요 반찬하기 싫지만 집밥 먹고싶을 때 요거랑 계란후라이 김만 있...
...,...,...,...,...,...
3,5132868,금돼지식당 통삽겹김치찌개,2022-03-22,0,워낙고기가유명한데라 찌개도궁금해서ㅋ
4,5132868,금돼지식당 통삽겹김치찌개,2022-03-22,0,김치찌개 간편하게 먹을 수 있어 좋아요
5,5132868,금돼지식당 통삽겹김치찌개,2022-03-22,0,첫구매인데 먹을만 햇어요
6,5132868,금돼지식당 통삽겹김치찌개,2022-03-22,0,금돼지식당 정말 좋아하는데 어제 김치찌개 먹고싶던 차에 컬리에도 드뎌 피코크 금돼지...


In [87]:
new.to_csv("[232-233] 크롤링.csv", index=False, encoding='utf-8-sig')

### 날짜 필터링하기

In [46]:
threshold_date = '2024.08.25'
threshold_date = datetime.strptime(threshold_date, '%Y.%m.%d')

review_df['date'] = pd.to_datetime(review_df['date'], format='%Y.%m.%d')
review_df = review_df[review_df['date'] <= threshold_date]

review_df

,product_id,product_type,date,help_num,review
0,5132868,김치 우동 어묵 전골 (리뉴얼),2023-11-05,40,진짜 사고 싶었던 삼진어묵 김치 어묵 전골입니다. \n컬리에서 일일특가로 완전 득템...
1,5132868,김치 우동 어묵 전골 (리뉴얼),2023-07-24,50,"캠핑가서 진짜 맛있게 먹었어요.\n\n식사용으로는 양이 부족할 것 같아서, 물놀이 ..."
2,5132868,김치 우동 어묵 전골 (리뉴얼),2023-06-19,39,저의 구매 목적은 뚜벅이 여행을 위해..짐이 많으면 안되니\n제일 간편하게 가져갈 ...
6,5132868,김치 우동 어묵 전골 (리뉴얼),2024-08-25,0,생각했던 김치 우동 맛이였어요 무난합니다
7,5132868,김치 우동 어묵 전골 (리뉴얼),2024-08-25,2,2인이서 드실거면 사이드 넉넉히 준비하세요!!\n우동만 드시기엔 두명은 부족..할 ...
...,...,...,...,...,...
3,5132868,김치 우동 어묵 전골,2022-10-14,0,김치우동 어묵 전골 삼진어묵 맛있으니 맛있을것같아요
4,5132868,김치 우동 어묵 전골,2022-10-14,0,선택했어요\n실수로 3개시켰지만 기대중입니다
5,5132868,김치 우동 어묵 전골,2022-10-13,0,처음 주문 해보아요
6,5132868,김치 우동 어묵 전골,2022-10-13,0,타 브랜거 먹다 세일하길래 주문해요


In [26]:
review_df = review_df.reset_index(drop=True)
review_df

,product_id,product_type,date,help_num,review
0,5097092,한우사골 양지곰탕,2024-06-28,9,간편식으로 먹으려고 한우사골 양지곰탕을 구매해 보았습니다.\n한우사골 양지곰탕의 조...
1,5097092,한우사골 양지곰탕,2023-08-01,36,밥 잘 안챙겨먹고 요리도 귀찮고 하기 힘든 스타일 이시라면 무조건 구매해 두세요.....
2,5097092,한우사골 양지곰탕,2023-04-08,36,한우사골 양지곰탕으로 \n사골떡만두칼국수🍜 만들어 먹었어요 ~ \n\n국물이 진하고...
3,5097092,한우사골 양지곰탕,2024-08-25,0,고기도 많고 애들이 맛있게 잘먹어요
4,5097092,한우사골 양지곰탕,2024-08-25,0,"곰탕 늘 쟁여두는 편인데, 이 제품도 나쁘지 않은 듯"
...,...,...,...,...,...
23603,5132868,부산깡통시장 유부주머니,2022-05-12,0,나들이용으로 구입했어오\n맛있길 기도합니다 ㅎㅎ
23604,5132868,부산깡통시장 유부주머니,2022-05-11,0,신랑이 좋아해서 구매합니다
23605,5132868,부산깡통시장 유부주머니,2022-05-11,0,유부주머니 넘 좋아하는데 기대되요
23606,5132868,부산깡통시장 유부주머니,2022-05-11,0,부산 여행 생각나서 구매해봤어요


### 중복 확인

In [150]:
review_df[review_df['product_id'] ==5110442].duplicated().sum()

79

In [ ]:
duplicates = review_df[review_df['product_id'] == 5110442].duplicated()
review_df[review_df['product_id'] == 5110442][duplicates]

### 리뷰 개수 맞는지 확인

In [21]:
review_df[review_df['product_id'] == 5061723].shape[0]

50295

In [29]:
count = 0

for id in 저장['product_id'].unique() :
    review_count = 저장[저장['product_id'] == id].shape[0]
    product_review_num = product_df[product_df['product_id'] == id]['review_num']
    
    if not product_review_num.empty and review_count != product_review_num.values[0]:
        count+=1
        print(id, review_count-product_review_num.values[0])
        
print("리뷰 수 안 맞는 상품 개수:", count)

5049304 2
5031816 -1
5070615 -1
5052864 1
5116405 2
5026469 1
5131413 1
5119615 -2290
1000047929 -11698
1000085968 7
5031054 1
5148360 5
5119471 -1
5044756 1
5097307 2
리뷰 수 안 맞는 상품 개수: 15


### 파일 저장하기

`[시작 index-끝 index] 크롤링.csv"` 형태

In [28]:
review_df.to_csv("[220-240] 일부_1 크롤링.csv", index=False, encoding='utf-8-sig')

### 파일 합치기

In [98]:
df_0_220 = pd.read_csv("[0-220] 크롤링.csv")
df_220_240 = pd.read_csv("[220-240] 크롤링.csv")
df_240_255 = pd.read_csv("[240-255] 크롤링.csv")
df_255_264 = pd.read_csv("[255-264] 크롤링.csv")
df_264_269 = pd.read_csv("[264-269] 크롤링.csv")

In [97]:
product_df[250:256]

,product_id,product_url,product_name,review_num
250,5128143,https://www.kurly.com/goods/5128143,[홍익궁중전통] 육개장 (2인분),22791
251,5058878,https://www.kurly.com/goods/5058878,[고래사] 프리미엄 어묵탕,22950
252,5110442,https://www.kurly.com/goods/5110442,[선물세트] 사미헌 불고기/맑은곰탕 선물세트,23047
253,5034225,https://www.kurly.com/goods/5034225,[워커힐] 명월관 갈비탕,24119
254,5055553,https://www.kurly.com/goods/5055553,[더오담] 콩비지찌개,24333
255,5097090,https://www.kurly.com/goods/5097090,[사미헌] 맑은 곰탕,25893


In [101]:
product_df['product_id'].nunique()

269

In [99]:
전체 = pd.concat([df_0_220, df_220_240,df_240_255, df_255_264, df_264_269])
전체 

,product_id,product_type,date,help_num,review
0,1000498619,양곰탕,2024-05-26,1,양곰탕 맛집에서 먹는 것보다 맛있어서 몇개 더 쟁였어요!\n\n[맛]\n보통 이런 ...
1,1000498619,양곰탕,2024-05-25,2,평상시에 LF푸드 제품을 좋아해요.\n한식 종류가 안나오는게 늘 아쉽긴했는데\n이번...
2,1000498619,양곰탕,2024-08-22,0,맛있어요 ㅎㅎ 재구매 할려구요 ㅎㅎ
3,1000498619,양곰탕,2024-08-22,0,개봉후 냄비에 부으니 기름 덩어리 분리되어 있으니 걷어내기 좋네요\n물 조금 더 추...
4,1000498619,양곰탕,2024-08-15,0,후기좋아서 사봤는데 정말 고기가 듬뿍들었고 팍팍하지않고 부들부들하며 고소해요\n근데...
...,...,...,...,...,...
240807,5026468,갈비탕,2022-11-27,0,다른 브랜드랑 비교해보려고 샀어요 맛있길 기대합니다
240808,5026468,갈비탕,2022-11-27,0,맛있어서 자주 구매해요ㆍ
240809,5026468,갈비탕,2022-11-27,0,너무맛있어요 고기도 질기지않고 아이들 먹기좋아요
240810,5026468,갈비탕,2022-11-27,0,갈비탕중 최고예요 맛있어요


In [100]:
전체['product_id'].nunique()

267

In [102]:
전체 = 전체.reset_index(drop=True)
전체

,product_id,product_type,date,help_num,review
0,1000498619,양곰탕,2024-05-26,1,양곰탕 맛집에서 먹는 것보다 맛있어서 몇개 더 쟁였어요!\n\n[맛]\n보통 이런 ...
1,1000498619,양곰탕,2024-05-25,2,평상시에 LF푸드 제품을 좋아해요.\n한식 종류가 안나오는게 늘 아쉽긴했는데\n이번...
2,1000498619,양곰탕,2024-08-22,0,맛있어요 ㅎㅎ 재구매 할려구요 ㅎㅎ
3,1000498619,양곰탕,2024-08-22,0,개봉후 냄비에 부으니 기름 덩어리 분리되어 있으니 걷어내기 좋네요\n물 조금 더 추...
4,1000498619,양곰탕,2024-08-15,0,후기좋아서 사봤는데 정말 고기가 듬뿍들었고 팍팍하지않고 부들부들하며 고소해요\n근데...
...,...,...,...,...,...
1581412,5026468,갈비탕,2022-11-27,0,다른 브랜드랑 비교해보려고 샀어요 맛있길 기대합니다
1581413,5026468,갈비탕,2022-11-27,0,맛있어서 자주 구매해요ㆍ
1581414,5026468,갈비탕,2022-11-27,0,너무맛있어요 고기도 질기지않고 아이들 먹기좋아요
1581415,5026468,갈비탕,2022-11-27,0,갈비탕중 최고예요 맛있어요


### 아예 접근이 안되는 페이지

In [498]:
for index in product_df['product_id'][:220]:
    if index not in df_0_220['product_id'].unique():
        print(index)

5055964
5007376
